In [ ]:
# Run setup_imports to enable importing from src/
%run ../src/setup_imports.py

In [ ]:
from src.utils.ibm import get_backend, get_backend_snapshot, run_and_wrap
from src.utils.python_utils import clean_datetimes
from src.utils.circuits import packed_chsh_circuit, phi_plus_meassured, compute_parallel_CHSH_scores
from qiskit_ibm_runtime import SamplerV2 as Sampler
from datetime import datetime
import json
import time
import pandas as pd

In [ ]:
# Parametrize experiment
reps = 1
delay_between_reps = 60 # seconds
max_duration = 10 * 60 * 60  # 10 hours in seconds
introspection_shots = 1024
main_circuit_shots = 1024
entanglement_threshold = 1.9

In [ ]:
backend = get_backend()

In [ ]:
# Main experiment loop

results = []

i = 0
while i < reps:
    i+=1
    entry = {}
    
    # --- Backend snapshot; need we do this more than once?---
    entry['backend_snapshot'] = get_backend_snapshot(backend)

    # --- Introspection phase ---
    print(f"{i}. Running introspection circuit")
    entry['introspection'] = run_and_wrap(packed_chsh_circuit(), backend, introspection_shots)
    entry['entanglement_score'] = compute_parallel_CHSH_scores(entry['introspection']["counts"])['CHSH_score']

    # --- Decision phase ---
    if entry['entanglement_score'] > entanglement_threshold:  
        print(f"{i}. Running main circuit - entanglement score was {entry['entanglement_score']}")
        entry['execution'] = run_and_wrap(phi_plus_meassured(), backend, main_circuit_shots)
    else:
        print(f"{i}. Skipping main circuit - entanglement score was {entry['entanglement_score']}")
    
    results.append(clean_datetimes(entry))

    if i < reps:
        time.sleep(delay_between_reps) # wait before next repetition 

# Save results
with open("experiment_results.json", "w") as f:
    json.dump(results, f, indent=2)

# Tabular summary
df = pd.DataFrame([{
    'introspection_created': r.get('introspection', {}).get('timestamps', {}).get('created'),
    'introspection_running': r.get('introspection', {}).get('timestamps', {}).get('running'),
    'introspection_finished': r.get('introspection', {}).get('timestamps', {}).get('finished'),
    'execution_created': r.get('execution', {}).get('timestamps', {}).get('created'),
    'execution_running': r.get('execution', {}).get('timestamps', {}).get('running'),
    'execution_finished': r.get('execution', {}).get('timestamps', {}).get('finished'),    
    'entanglement_score': r.get('entanglement_score'),
    'execution_counts': json.dumps(r.get('execution', {}).get('counts')) if r.get('execution') else None
} for r in results])
df.to_csv("experiment_summary.csv", index=False)

In [ ]:
entry

In [ ]:
entry['execution']['counts']